In [1]:
import pickle
import numpy as np
import pandas as pd
from libraries.Crawler import crawler


with open("./logs/model.pickle","rb") as f:
    model=pickle.load(f)
    
    
feat_cols=[
        'Temperature',
        'Humidity',
        'Mean Sea Level Pressure  [MSL]',
        'Wind Speed  [10 m above gnd]',
        'High Cloud Cover  [high cld lay]',
        'Wind Direction  [10 m above gnd]',
        'Wind Gust  [sfc]'
]

grid_dev=[(1,3),(0,0),(3,2),(2,1)]
eucledine=lambda a,b,c,d:round(((a-c)**2+(b-d)**2)**0.5,2)

def get_contribution_vec(x,y):
    try:
        device=grid_dev.index((x,y))
        #print('It is a device')
        vec=[0,0,0,0]
        vec[device]=1
    except ValueError:
        #print('Not a device')
        vec=[0,0,0,0]
        for i,(a,b) in enumerate(grid_dev):
            vec[i]=eucledine(a,b,x,y)
    return vec


grid=[]
for i in range(4):
    for j in range(4):
        grid.append(get_contribution_vec(i,j))



def Interpolate(dev,rc):

    #interpolating w.r.to distance vec for each point.

    interpol=[]
    for g in grid:
        x=1/np.array(g).reshape(-1,1)
        x[x == np.inf] = 0

        interpol.append((x*dev).sum(axis=0)/np.sum(x))

    #Interpolated features

    grid_feat=pd.DataFrame(interpol)[feat_cols]

    #RandomForest Predictions

    grid_wise_pred=rc.predict(grid_feat)
    
    return np.array(grid_wise_pred,dtype="int").reshape(4,4).tolist()


In [2]:
from flask import Flask,jsonify
from flask_cors import cross_origin
app = Flask(__name__)

@cross_origin
@app.route('/<date>/<hour>')
def fun(date='2020-12-19',hour='11'):
    df=crawler(date+' '+hour)[feat_cols]
    pred=Interpolate(df,model)
    
    data=\
        [
          {
            "z": pred,
            "type": 'heatmap'
          }
        ]
    return jsonify(data)

In [3]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Dec/2020 04:42:34] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [30/Dec/2020 04:42:34] "GET /favicon.ico HTTP/1.1" 404 -
C:\Users\ME\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: divide by zero encountered in true_divide
127.0.0.1 - - [30/Dec/2020 04:44:53] "GET /2020-12-25/11 HTTP/1.1" 200 -


In [ ]:
#APP Done